In [ ]:
import skel_features
from meshparty import meshwork
from caveclient import CAVEclient
from cloudpathlib import AnyPath as Path
import cloudfiles
from io import BytesIO
import tqdm.autonotebook as tqdm

import numpy as np


client = CAVEclient('minnie65_public')
client.version = 1300

In [ ]:
int(all_files[0].name.split(".")[0])

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

In [ ]:
df.columns = ["id", 'structure', "x", "y", "z", "radius", "parent"]

In [ ]:
df.structure.value_counts()

In [ ]:
skel_dir = Path("gs://csm-skel/skel-test/")
swc_dir = Path("gs://microns-static-links/skel/swc/proofread/")

cf = cloudfiles.CloudFiles("gs://microns-static-links/skel/swc/proofread/")

In [ ]:
tbl_qry = client.materialize.tables
pf_df = tbl_qry.proofreading_status_and_strategy.get_all()
pf_df.query('status_axon == "t"')

In [ ]:
def convert_to_swc(root_id, skel_dir, cf):
    try:
        nrn = skel_features.io_utils.load_root_id(root_id, skel_dir, 0.1)
        dend_inds = nrn.anno.compartment.df.query('compartment == 3').mesh_ind
        axon_inds = nrn.anno.compartment.df.query('compartment == 2').mesh_ind

        nrn.anno.add_annotations('is_dendrite', dend_inds, mask=True, overwrite=True)
        nrn.anno.add_annotations('is_axon', axon_inds, mask=True, overwrite=True)

        with BytesIO() as bio:
            nrn.export_to_swc(
                bio,
                resample_spacing=1000,
                axon_label='is_axon',
                dendrite_label='is_dendrite',
                soma_label='is_soma',
                radius=nrn.anno.segment_properties.df['r_eff'].values,
            )
            bio.seek(0)
            cf.put(f'{root_id}.swc', bio.read())
        return True
    except Exception as e:
        print(f"Error processing root_id {root_id}: {e}")
        return False
    

In [ ]:
root_id = pf_df.query('status_axon == "t"')['pt_root_id'].values[30]
convert_to_swc(root_id, skel_dir, cf)

In [ ]:
from joblib import Parallel, delayed

success = Parallel(n_jobs=16)(
    delayed(convert_to_swc)(root_id, skel_dir, cf)
    for root_id in pf_df.query('status_axon == "t"')['pt_root_id'].values
)

In [ ]:
import pcg_skel

Do singlet cell that wasn't in the bulk hydration

In [ ]:
rid = 864691136119883288
nrn = pcg_skel.service.get_meshwork_from_client(
    rid,
    client,
    synapses=True,
    restore_properties=True,
)

dend_inds = nrn.anno.compartment.df.query('compartment == 3').mesh_ind
axon_inds = nrn.anno.compartment.df.query('compartment == 2').mesh_ind
soma_inds = nrn.anno.compartment.df.query('compartment == 1').mesh_ind

nrn.anno.add_annotations('is_dendrite', dend_inds, mask=True, overwrite=True)
nrn.anno.add_annotations('is_axon', axon_inds, mask=True, overwrite=True)
nrn.anno.add_annotations('is_soma', soma_inds, mask=True, overwrite=True)

with BytesIO() as bio:
    nrn.export_to_swc(
        bio,
        resample_spacing=1000,
        axon_label='is_axon',
        dendrite_label='is_dendrite',
        soma_label='is_soma',
        radius=nrn.anno.segment_properties.df['r_eff'].values,
    )
    bio.seek(0)
    cf.put(f'{rid}.swc', bio.read())


Do thalamic axons

In [ ]:
Parallel?

In [ ]:
thal_axon = [864691135279153313, 864691135489461050, 864691135561626081, 864691135572519149, 864691135700619810, 864691135738851953, 864691135874798734, 864691135947132769, 864691135947139681, 864691136419064727, 864691136618899341, 864691136967639758]

In [ ]:
for rid in tqdm.tqdm(thal_axon[10:]):
    nrn = pcg_skel.service.get_meshwork_from_client(
        rid,
        client,
        synapses=True,
        restore_properties=True,
        )

    axon_inds = np.arange(len(nrn.vertices))

    nrn.anno.add_annotations('is_axon', axon_inds, mask=True, overwrite=True)

    with BytesIO() as bio:
        nrn.export_to_swc(
            bio,
            resample_spacing=1000,
            axon_label='is_axon',
            radius=nrn.anno.segment_properties.df['r_eff'].values,
        )
        bio.seek(0)
        cf.put(f'{rid}.swc', bio.read())

Now do dendrite-only for all neurons

In [ ]:
root_id = pf_df.query('status_axon == "t"')['pt_root_id'].values[800]

In [ ]:
nrn = skel_features.io_utils.load_root_id(root_id, skel_dir)
nrn.apply_mask(nrn.anno.is_dendrite.mesh_mask)


In [ ]:
nrn.export_to_swc?

In [ ]:
nrn = skel_features.io_utils.load_root_id(root_id, skel_dir)
nrn.apply_mask(nrn.anno.is_dendrite.mesh_mask)

nrn.export_to_swc(
    'dend_test.swc',
    resample_spacing=1000,
    soma_label='is_soma',
    radius=nrn.anno.segment_properties.df['r_eff'].values,
)

In [ ]:
def convert_dendrite_to_swc(root_id, skel_dir, cf):
    try:
        nrn = skel_features.io_utils.load_root_id(root_id, skel_dir)
        nrn.apply_mask(nrn.anno.is_dendrite.mesh_mask)

        with BytesIO() as bio:
            nrn.export_to_swc(
                bio,
                resample_spacing=1000,
                dendrite_label='is_dendrite',
                radius=nrn.anno.segment_properties.df['r_eff'].values,
            )
            bio.seek(0)
            cf.put(f'{root_id}.swc', bio.read())
        return True
    except Exception as e:
        print(f"Error processing root_id {root_id}: {e}")
        return False
    

In [ ]:
swc_dir = Path("gs://microns-static-links/skel/swc/proofread/")
all_files = list(swc_dir.glob("*.swc"))
pf_root_ids = [int(f.name.split(".")[0]) for f in all_files]
filenames = [str(f).replace("gs://", "https://storage.googleapis.com/") for f in all_files]
pf_df = pd.DataFrame(
    {
        "root_id": pf_root_ids,
        "filepath": filenames
    }
).to_csv("proofread_manifest.csv", index=False)
swc_dir = Path("gs://microns-static-links/skel/swc/dendrite/")
all_files = list(swc_dir.glob("*.swc"))
dend_root_ids = [int(f.name.split(".")[0]) for f in all_files]
filenames = [str(f).replace("gs://", "https://storage.googleapis.com/") for f in all_files]
dend_df = pd.DataFrame(
    {
        "root_id": dend_root_ids,
        "filepath": filenames
    }
).to_csv("dendrite_manifest.csv", index=False)


In [ ]:
pd.read_csv(
    pf_df.filepath[0],
    sep=" ",
    header=None,
    names=["id", 'type', "x", "y", "z", "radius", "parent"],
)